In [ ]:
import time
import torch
while True:
    a = torch.zeros([10, 10, 10], dtype = torch.float32, device = 'cuda:3', requires_grad = False)
    b = a * a
    print(b[0, 0, 0])
    time.sleep(300)

tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)

In [2]:
import defdevice
defdevice.force_device('cuda:3')
import metrics
import nns
import immaskdataset
import importlib
importlib.reload(immaskdataset)
import imcapdataset
import embdataset
importlib.reload(embdataset)
import embedders
importlib.reload(embedders)
import losses
import imageops
import segmodel

In [3]:
image_ds = embdataset.RAMLoadDataset('COCOM2F_ALL_d8', load_amount = 1000000, norm = True)
text_ds = embdataset.RAMLoadDataset('COCOCLIP_ALL', load_amount = 1000000)

Estimated necessary RAM for loading - 294912000000 bytes
Warning, requested to load 1000000 samples, but dataset contained only 566747
Estimated necessary RAM for loading - 4096000000 bytes
Warning, requested to load 1000000 samples, but dataset contained only 566747


In [4]:
model = nns.Linear(text_ds.embedding_shape, image_ds.embedding_shape)

In [5]:
def_device = 'cuda:3'

image_ds.reset()
text_ds.reset()

import torch
import torch.nn.functional

batch_count = 200000
batch_size = 128
optim = torch.optim.Adam(model.parameters, lr = 0.003)

image_noise = 0.07 / 10
text_noise = 0.15

for batch_idx in range(batch_count):
    ibatch = image_ds.get_batch(batch_size).to(def_device)
    tbatch = text_ds.get_batch(batch_size).to(def_device)
    
    ibatch = ibatch + torch.rand_like(ibatch) * image_noise
    tbatch = tbatch + torch.rand_like(tbatch) * text_noise
    
    tbatch = model.forward(tbatch)
    
    neg_ibatch, neg_tbatch = losses.sample_negatives(ibatch, tbatch, batch_size)
    
    prods = losses.get_products(ibatch, tbatch)
    probs = torch.sigmoid(prods)
    
    neg_prods = losses.get_products(neg_ibatch, neg_tbatch)
    neg_probs = torch.sigmoid(neg_prods)
    
    loss_positive = losses.max_pressure_loss(probs, 1.0)
    loss_negative = losses.max_pressure_loss(neg_probs, 0.0)
    
    loss = (loss_positive + loss_negative) / 2
    
    loss.backward()
    optim.step()
    optim.zero_grad()
    
    if batch_idx % 1000 == 0:
        print(batch_idx, loss_positive, loss_negative)

0 tensor(0.2275, grad_fn=<MeanBackward0>) tensor(0.2721, grad_fn=<MeanBackward0>)
1000 tensor(0.1566, grad_fn=<MeanBackward0>) tensor(0.1153, grad_fn=<MeanBackward0>)
2000 tensor(0.1295, grad_fn=<MeanBackward0>) tensor(0.1703, grad_fn=<MeanBackward0>)
3000 tensor(0.0995, grad_fn=<MeanBackward0>) tensor(0.1476, grad_fn=<MeanBackward0>)
4000 tensor(0.1242, grad_fn=<MeanBackward0>) tensor(0.0927, grad_fn=<MeanBackward0>)
5000 tensor(0.0998, grad_fn=<MeanBackward0>) tensor(0.1124, grad_fn=<MeanBackward0>)
6000 tensor(0.1049, grad_fn=<MeanBackward0>) tensor(0.1331, grad_fn=<MeanBackward0>)
7000 tensor(0.0768, grad_fn=<MeanBackward0>) tensor(0.1274, grad_fn=<MeanBackward0>)
8000 tensor(0.0741, grad_fn=<MeanBackward0>) tensor(0.2112, grad_fn=<MeanBackward0>)
9000 tensor(0.0681, grad_fn=<MeanBackward0>) tensor(0.1338, grad_fn=<MeanBackward0>)
10000 tensor(0.0959, grad_fn=<MeanBackward0>) tensor(0.1757, grad_fn=<MeanBackward0>)
11000 tensor(0.1132, grad_fn=<MeanBackward0>) tensor(0.1391, grad_f

96000 tensor(0.0800, grad_fn=<MeanBackward0>) tensor(0.0806, grad_fn=<MeanBackward0>)
97000 tensor(0.0567, grad_fn=<MeanBackward0>) tensor(0.1219, grad_fn=<MeanBackward0>)
98000 tensor(0.0847, grad_fn=<MeanBackward0>) tensor(0.0987, grad_fn=<MeanBackward0>)
99000 tensor(0.0925, grad_fn=<MeanBackward0>) tensor(0.0903, grad_fn=<MeanBackward0>)
100000 tensor(0.0568, grad_fn=<MeanBackward0>) tensor(0.1040, grad_fn=<MeanBackward0>)
101000 tensor(0.0794, grad_fn=<MeanBackward0>) tensor(0.1321, grad_fn=<MeanBackward0>)
102000 tensor(0.0863, grad_fn=<MeanBackward0>) tensor(0.0950, grad_fn=<MeanBackward0>)
103000 tensor(0.0780, grad_fn=<MeanBackward0>) tensor(0.1055, grad_fn=<MeanBackward0>)
104000 tensor(0.1020, grad_fn=<MeanBackward0>) tensor(0.0941, grad_fn=<MeanBackward0>)
105000 tensor(0.0889, grad_fn=<MeanBackward0>) tensor(0.0863, grad_fn=<MeanBackward0>)
106000 tensor(0.0685, grad_fn=<MeanBackward0>) tensor(0.1293, grad_fn=<MeanBackward0>)
107000 tensor(0.0768, grad_fn=<MeanBackward0>) 

191000 tensor(0.0590, grad_fn=<MeanBackward0>) tensor(0.1050, grad_fn=<MeanBackward0>)
192000 tensor(0.0426, grad_fn=<MeanBackward0>) tensor(0.1103, grad_fn=<MeanBackward0>)
193000 tensor(0.0366, grad_fn=<MeanBackward0>) tensor(0.0929, grad_fn=<MeanBackward0>)
194000 tensor(0.0748, grad_fn=<MeanBackward0>) tensor(0.1129, grad_fn=<MeanBackward0>)
195000 tensor(0.0950, grad_fn=<MeanBackward0>) tensor(0.1180, grad_fn=<MeanBackward0>)
196000 tensor(0.0787, grad_fn=<MeanBackward0>) tensor(0.1417, grad_fn=<MeanBackward0>)
197000 tensor(0.0754, grad_fn=<MeanBackward0>) tensor(0.0936, grad_fn=<MeanBackward0>)
198000 tensor(0.0541, grad_fn=<MeanBackward0>) tensor(0.1069, grad_fn=<MeanBackward0>)
199000 tensor(0.0957, grad_fn=<MeanBackward0>) tensor(0.0573, grad_fn=<MeanBackward0>)


In [6]:
model.save("CSTableModel")

In [7]:
image_emb = embedders.M2FImageEmbedder()
text_emb = embedders.CLIPTextEmbedder()

In [8]:
dataset_results_normed = {}

In [9]:
dataset_results = {}

In [10]:
ds = immaskdataset.RUGDDataset()
res, res_dict = metrics.test(model, ds, image_emb, text_emb, amount=2000)
dataset_results['RUGD25_2000'] = {'summary': res, 'metrics': res_dict}
print(res)
import segmodel
smodel = segmodel.CSModel(image_emb, text_emb, model)
ds = immaskdataset.ADEDataset(split = 'validation')
res, res_dict = metrics.test(model, ds, image_emb, text_emb, amount=ds.sample_count)
dataset_results['ADE'] = {'summary': res, 'metrics': res_dict}
print(res)
ds = immaskdataset.RUGDDataset()
res, res_dict = metrics.test(model, ds, image_emb, text_emb, amount=ds.sample_count)
dataset_results['RUGD25'] = {'summary': res, 'metrics': res_dict}
print(res)
smodel = segmodel.CSModel(image_emb, text_emb, model)
ds = immaskdataset.RUGD6Dataset()
res, res_dict = metrics.test_segmodel(smodel, ds, amount = ds.sample_count, multilabel = True)
dataset_results['RUGD6'] = {'summary': res, 'metrics': res_dict}
print(res)

/tf/FreshCLIPSchitzo/metrics.py:181: RuntimeWarning: divide by zero encountered in true_divide
  res = 2 / (r + p)


MSE over everything - 0.06626384956560517
Pixel accuracy - 0.7257817600422833
Mean recall - 0.23577301340645002
Mean precision - 0.29263846721457565
Mean IoU - 0.16827700158133518
Mean F1 - inf

Small Recall: 0.0 - rock-bed
Small Recall: 0.0 - log
Small Recall: 0.0 - bicycle
Small Recall: 0.0 - sign
Small Recall: 0.0 - rock

Big   Recall: 0.9803078094658623 - tree
Big   Recall: 0.9135853847580554 - grass
Big   Recall: 0.7979691017183884 - asphalt
Big   Recall: 0.7723234953396985 - building
Big   Recall: 0.6383509349403677 - sand

Small Precision: 0.0 - rock-bed
Small Precision: 0.0 - log
Small Precision: 0.0 - bicycle
Small Precision: 0.0 - sign
Small Precision: 0.0 - rock

Big   Precision: 0.9737713666138548 - sky
Big   Precision: 0.8856653369641795 - grass
Big   Precision: 0.8551822712598048 - tree
Big   Precision: 0.8090287308763009 - building
Big   Precision: 0.8047149826929655 - vehicle

Small IoU: 0.0 - rock-bed
Small IoU: 0.0 - log
Small IoU: 0.0 - bicycle
Small IoU: 0.0 - sign


In [11]:
dataset_results_normed = dataset_results

In [12]:
dataset_results = {}

# ADE

In [48]:
ds = immaskdataset.ADEDataset(split = 'validation')
res, res_dict = metrics.test(model, ds, image_emb, text_emb, amount=ds.sample_count)
dataset_results['ADE'] = {'summary': res, 'metrics': res_dict}
print(res)

MSE over everything - 0.07558230294071577
Pixel accuracy - 0.2568257491257151
Mean recall - 0.14995441547042238
Mean precision - 0.10270758686196382
Mean IoU - 0.052870771265971755
Mean F1 - inf

Small Recall: 0.0 - chest of drawers, chest, bureau, dresser
Small Recall: 0.0 - blind, screen
Small Recall: 0.0 - stage
Small Recall: 0.0 - ship
Small Recall: 0.0 - stool

Big   Recall: 0.8465935729488954 - sky
Big   Recall: 0.8281409418515379 - toilet, can, commode, crapper, pot, potty, stool, throne
Big   Recall: 0.7255045889254292 - windowpane, window 
Big   Recall: 0.7236680210596992 - food, solid food
Big   Recall: 0.7235471594603602 - building, edifice

Small Precision: 0.0 - chest of drawers, chest, bureau, dresser
Small Precision: 0.0 - blind, screen
Small Precision: 0.0 - stage
Small Precision: 0.0 - ship
Small Precision: 0.0 - stool

Big   Precision: 0.9789173047307913 - sky
Big   Precision: 0.8694735779132307 - person, individual, someone, somebody, mortal, soul
Big   Precision: 0.

# RUGD25

In [64]:
ds = immaskdataset.RUGDDataset()
res, res_dict = metrics.test(model, ds, image_emb, text_emb, amount=ds.sample_count)
dataset_results['RUGD25'] = {'summary': res, 'metrics': res_dict}
print(res)

MSE over everything - 0.1031723337874759
Pixel accuracy - 0.6734464118476254
Mean recall - 0.2863997394932551
Mean precision - 0.3912725694440828
Mean IoU - 0.19713455414905823
Mean F1 - 462.73733984345324

Small Recall: 0.00015398490947645156 - void
Small Recall: 0.00024268156245165571 - log
Small Recall: 0.0016522441071951815 - rock
Small Recall: 0.002347071148156318 - dirt
Small Recall: 0.00738284142013208 - pole

Big   Recall: 0.9748001200511486 - tree
Big   Recall: 0.880728929842395 - grass
Big   Recall: 0.7786546880548233 - building
Big   Recall: 0.7419238683667837 - asphalt
Big   Recall: 0.6169898734288254 - rock-bed

Small Precision: 2.6543245989731168e-05 - void
Small Precision: 0.003474721464369344 - sand
Small Precision: 0.004467605527474878 - dirt
Small Precision: 0.019396487693354454 - sign
Small Precision: 0.027169238282918517 - rock

Big   Precision: 0.9801543974012605 - sky
Big   Precision: 0.9192531594115406 - vehicle
Big   Precision: 0.8794240931041164 - grass
Big   P

# RUGD6

In [50]:
import segmodel
import importlib
smodel = segmodel.CSModel(image_emb, text_emb, model)
ds = immaskdataset.RUGD6Dataset()
res, res_dict = metrics.test_segmodel(smodel, ds, amount = ds.sample_count, multilabel = True)
dataset_results['RUGD6'] = {'summary': res, 'metrics': res_dict}
print(res)

MSE over everything - 0.049824993413100614
Pixel accuracy - 0.8582867321225692
Mean recall - 0.6564863248095504
Mean precision - 0.7311429532468052
Mean IoU - 0.5238154067594313
Mean F1 - 1.5661216639492839

Small Recall: 0.20270176946914265 - ['water']
Small Recall: 0.5163156067130394 - ['void', 'sky', 'sign']
Small Recall: 0.6203351026560234 - ['rock-bed', 'rock']
Small Recall: 0.806176164530557 - ['asphalt', 'concrete']
Small Recall: 0.8776506087390114 - ['dirt', 'sand', 'grass', 'gravel', 'mulch']

Big   Recall: 0.9157386967495282 - ['tree', 'pole', 'vehicle', 'container/generic-object', 'building', 'log', 'bicycle', 'person', 'fence', 'bush', 'bridge', 'picnic-table']
Big   Recall: 0.8776506087390114 - ['dirt', 'sand', 'grass', 'gravel', 'mulch']
Big   Recall: 0.806176164530557 - ['asphalt', 'concrete']
Big   Recall: 0.6203351026560234 - ['rock-bed', 'rock']
Big   Recall: 0.5163156067130394 - ['void', 'sky', 'sign']

Small Precision: 0.3006084396830373 - ['rock-bed', 'rock']
Small

# Extra Verification

In [65]:
ds = immaskdataset.RUGDDataset()
res, res_dict = metrics.test(model, ds, image_emb, text_emb, amount=2000)
dataset_results['RUGD25_2000'] = {'summary': res, 'metrics': res_dict}
print(res)

MSE over everything - 0.09206092617848176
Pixel accuracy - 0.7235711852536998
Mean recall - 0.22381532790552455
Mean precision - 0.2921900011245345
Mean IoU - 0.16289432068636744
Mean F1 - inf

Small Recall: 0.0 - rock-bed
Small Recall: 0.0 - log
Small Recall: 0.0 - bicycle
Small Recall: 0.0 - sign
Small Recall: 0.0 - rock

Big   Recall: 0.9765166329920218 - tree
Big   Recall: 0.9089175784231595 - grass
Big   Recall: 0.7887727153850098 - asphalt
Big   Recall: 0.7724616388478827 - building
Big   Recall: 0.5201318359127198 - sky

Small Precision: 0.0 - rock-bed
Small Precision: 0.0 - log
Small Precision: 0.0 - bicycle
Small Precision: 0.0 - sign
Small Precision: 0.0 - rock

Big   Precision: 0.9791832704927462 - sky
Big   Precision: 0.8923233799142923 - grass
Big   Precision: 0.8564930169550341 - tree
Big   Precision: 0.8094128631022091 - building
Big   Precision: 0.7897672874630784 - vehicle

Small IoU: 0.0 - rock-bed
Small IoU: 0.0 - log
Small IoU: 0.0 - bicycle
Small IoU: 0.0 - sign
Sm

/tf/FreshCLIPSchitzo/metrics.py:181: RuntimeWarning: divide by zero encountered in true_divide
  res = 2 / (r + p)


In [13]:
lseg_results = {'ADE': {'Accuracy': 0.667, 'Recall Mean': 0.399, 'Precision Mean': 0.408, 'IoU Mean': 0.274},
                'RUGD25': {'Accuracy': 0.703, 'Recall Mean': 0.411, 'Precision Mean': 0.440, 'IoU Mean': 0.294},
                'RUGD6': {'Accuracy': 0.836, 'Recall Mean': 0.709, 'Precision Mean': 0.804, 'IoU Mean': 0.600}}

In [14]:
import latexstuff
import numpy as np
dsname = ['ADE', 'RUGD25', 'RUGD6']
for name in dsname:
    cds = dataset_results_normed[name]['metrics']
    def extract(x):
         return np.round(np.array([x['Accuracy'], x['Recall Mean'], x['Precision Mean'], x['IoU Mean']]) * 1000) / 10
    
    ours = extract(cds)
    lseg = extract(lseg_results[name])
    fulldata = np.stack([ours, lseg], axis = 0)
    xlabels = ['Pixel Accuracy', 'Mean Recall', 'Mean Precision', 'Mean IoU']
    ylabels = ['Ours', 'LSeg']
    print(latexstuff.make_table(fulldata, xlabels, ylabels))

\begin{tabular}{l|cccc}
\toprule
 & Pixel Accuracy & Mean Recall & Mean Precision & Mean IoU \\
\midrule
Ours & 28.2 & 14.9 & 9.7 & 5.3 \\
LSeg & 66.7 & 39.9 & 40.8 & 27.4 \\
\bottomrule
\end{tabular}
\begin{tabular}{l|cccc}
\toprule
 & Pixel Accuracy & Mean Recall & Mean Precision & Mean IoU \\
\midrule
Ours & 67.5 & 29.8 & 37.1 & 20.0 \\
LSeg & 70.3 & 41.1 & 44.0 & 29.4 \\
\bottomrule
\end{tabular}
\begin{tabular}{l|cccc}
\toprule
 & Pixel Accuracy & Mean Recall & Mean Precision & Mean IoU \\
\midrule
Ours & 87.1 & 65.1 & 72.6 & 54.0 \\
LSeg & 83.6 & 70.9 & 80.4 & 60.0 \\
\bottomrule
\end{tabular}
